In [25]:
#!pip install msgpack 
import cython
import msgpack
import os
import numpy as np
import keras
import pandas
import tensorflow as tf
import sys

from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import backend as K

#os.getcwd()

In [2]:
# Load in Story
storyfile = "adventuresHF.txt"
raw_text = open(storyfile, encoding = "utf8").read()

raw_text = raw_text.lower()

In [8]:
## Check TF ##
# with tf.device('/gpu:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)

# with tf.Session() as sess:
#     print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
## Check TF ##
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [4]:
# Convert Char to Integer 
chars = sorted(list(set(raw_text)))
char2int = dict((c,i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print(n_chars, n_vocab)

567146 51


In [35]:
# Structure Training Set
seqlength = 100
dataX = []
dataY = []

for i in range(0, n_chars - seqlength, 1):
    seq_in = raw_text[i:i + seqlength]
    seq_out = raw_text[i + seqlength]
    dataX.append([char2int[char] for char in seq_in])
    dataY.append(char2int[seq_out])
n_patterns = len(dataX)
print("Total_Patterns:", n_patterns)

Total_Patterns: 567046


In [6]:
# Transform X [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seqlength, 1))

# Normalize
X = X/float(n_vocab)

# OneHot Encoding
y = np_utils.to_categorical(dataY)

In [7]:
### LSTM Model ### --- no test set
neurons = 256
model = Sequential()
model.add(LSTM(neurons, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [10]:
### Fit Model ###
model.fit(X, y, epochs = 10, batch_size = 512)

Epoch 1/10
567046/567046 [==============================] - 249s 439us/step - loss: 2.5008
Epoch 2/10
567046/567046 [==============================] - 255s 450us/step - loss: 2.4385
Epoch 3/10
567046/567046 [==============================] - 215s 379us/step - loss: 2.3782s - loss: 2.
Epoch 4/10
567046/567046 [==============================] - 202s 356us/step - loss: 2.3252
Epoch 5/10
567046/567046 [==============================] - 202s 356us/step - loss: 2.2816
Epoch 6/10
567046/567046 [==============================] - 202s 357us/step - loss: 2.2453
Epoch 7/10
567046/567046 [==============================] - 207s 364us/step - loss: 2.2136
Epoch 8/10
567046/567046 [==============================] - 204s 360us/step - loss: 2.1847
Epoch 9/10
567046/567046 [==============================] - 203s 358us/step - loss: 2.1595
Epoch 10/10
567046/567046 [==============================] - 203s 358us/step - loss: 2.1373


In [15]:
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [43]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\""), ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"
ed ano she tay and says:

“why, thet sii tood ano the tame and soe then the was a lintte th toed to the ware and saed ie was a good aedins and saed in wher they was anl she taye to the whrl whsh the sioe and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would be a coae and she was a lintte then the was a coas and the wirl would